<h3>Implementation of "Feature Selection for High Dimensional Data: A Fast Correlation-Based Filter Solution"</h3>

In [1]:
import numpy as np 
from math import log
import random

In [2]:
# Given a 1-dimensional list, return the information entropy of the list
# H(X)
def entropy_single(X):
    X_set = set(X)
    number_of_entry = len(X)
    entropy = 0
    for x in X_set:
        number_of_occurence = X.count(x)
        P_x = number_of_occurence/number_of_entry
        entropy -= (P_x)*log(P_x)
    return entropy

In [3]:
# Given two 1-dimensional list, it returns the entropy of the first list condition on the second list
# H(X|Y)
def entropy_conditional(X,Y):
    if(len(X)!=len(Y)):
        raise Exception("Dimension of both input must be the same")
    number_of_entry = len(Y)
    Y_set = set(Y)   
    entropy = 0
    for y in Y_set:
        number_of_occurence = Y.count(y)
        P_y = number_of_occurence/number_of_entry
        index_filter = [index for index, element in enumerate(Y) if element == y]
        X_filtered = [X[index] for index in index_filter]
        entropy -= P_y * (-entropy_single(X_filtered))
    return entropy

In [4]:
def information_gain(X,Y):
    return entropy_single(X)-entropy_conditional(X,Y)

In [5]:
def symmetrical_uncertainty(X,Y):
    return 2*information_gain(X,Y)/(entropy_single(X)+entropy_single(Y))

In [6]:
# S is an array with format [feature_index, SU, keep_data] for each entry
# F_p is an element in the array S
def get_next_element(S, F_p):
    S_size = len(S)
    next_index = S.index(F_p)
    while(next_index<S_size):
        next_index += 1
        if(next_index==S_size):
            return None
        elif(S[next_index][2]==1):
            return S[next_index]
        else:
            continue
    return None
    
def get_first_element(S):
    if len(S)==0:
        return None
    else:
        return S[0]
def remove_element(S, F_p):
    remove_index = S.index(F_p)
    S[remove_index][2] = 0
    return S

In [13]:
# X[i] = ith feature ; X[i][j] = jth entry of ith feature ; y[i] = ith entry of class list
# Conduct Fast Correlation-Based Filter on X, y
def FCBF(X, y, threshold = 0.1):
    if(len(X)==0):
        raise Exception("Input cannot be zero")
    elif(len(X[0])!=len(y)):
        raise Exception("Dimension of both input must be the same")
    number_of_feature = len(X)
    S_list = []
    for feature_index in range(number_of_feature):
        SU = symmetrical_uncertainty(X[feature_index], y)
        if(SU>=threshold):
             S_list.append([feature_index, SU, 1])
    S_list_length = len(S_list)
    S_list.sort(key= lambda x: -x[1])
    F_p = get_first_element(S_list)
    while F_p!=None:
        F_q = get_next_element(S_list, F_p)
        while(F_q!=None):
            F_q_dash = F_q
            SU_pq = symmetrical_uncertainty(X[F_q[0]], X[F_p[0]])
            if(SU_pq >= F_q[1]):
                S_list = remove_element(S_list, F_q)
                F_q = get_next_element(S_list, F_q_dash)
            else:
                F_q = get_next_element(S_list, F_q)
        F_p = get_next_element(S_list,F_p)
    X_filtered = []
    filtered_index = []
    for element in S_list:
        if(element[2]==1):
            X_filtered.append(X[element[0]])
            filtered_index.append(element[0])
        else:
            continue
    [element for element in S_list if ]
    return X_filtered, filtered_index

UCI Machine Learning Repository: Molecular Biology (Splice-junction Gene Sequences) Data Set 

In [14]:
with open("data.txt", "r") as lines:
    data = []
    for line in lines:
        data.append([x.strip() for x in line.split(',')])

In [15]:
for index,datum in enumerate(data):
    data[index] += list(datum[2])
    del data[index][2]

In [16]:
data_transpose = [list(i) for i in zip(*data)]
splice_class = data_transpose[0]
del data_transpose[0]
data_input = data_transpose

In [22]:
FCBF(data_input, splice_class, threshold=0.0)[1]

[30,
 29,
 31,
 32,
 35,
 28,
 33,
 34,
 25,
 24,
 23,
 20,
 19,
 21,
 18,
 17,
 16,
 9,
 14,
 6,
 12,
 55]